In [2]:
import os
import sys
import logging
from argparse import Namespace
import torch
import torchvision.transforms as T

os.environ["LOCAL_RANK"] = "1"
sys.path.append("Swin-Transformer")
from config import get_config
from models import build_model
from utils_simmim import load_pretrained

logger = logging.getLogger()

config_path = "Swin-Transformer/configs/hydro/hydro_simmim_pretrain_swinv2_base_img256_window16_800ep.yaml"
args = Namespace(cfg=config_path, opts=None)
config = get_config(args)
config

Tutel has not been installed. To use Swin-MoE, please install Tutel; otherwise, just ignore this.
=> merge config from Swin-Transformer/configs/hydro/hydro_simmim_pretrain_swinv2_base_img256_window16_800ep.yaml


CfgNode({'BASE': [''], 'DATA': CfgNode({'BATCH_SIZE': 128, 'DATA_PATH': '/workspace/storage/data/hydro/images/', 'DATASET': 'imagenet', 'IMG_SIZE': 256, 'INTERPOLATION': 'bicubic', 'ZIP_MODE': False, 'CACHE_MODE': 'part', 'PIN_MEMORY': True, 'NUM_WORKERS': 8, 'MASK_PATCH_SIZE': 32, 'MASK_RATIO': 0.6, 'BANDS': 'all', 'CHANNELS': 12, 'MEAN': [340.76769064, 429.9430203, 614.21682446, 590.23569706, 950.68368468, 1792.46290469, 2075.46795189, 2218.94553375, 2266.46036911, 2246.0605464, 1594.42694882, 1009.32729131], 'STD': [554.81258967, 572.41639287, 582.87945694, 675.88746967, 729.89827633, 1096.01480586, 1273.45393088, 1365.45589904, 1356.13789355, 1302.3292881, 1079.19066363, 818.86747235]}), 'MODEL': CfgNode({'TYPE': 'swinv2', 'NAME': 'hydro_simmim_pretrain', 'PRETRAINED': 'Swin-Transformer/output/hydro_simmim_pretrain1/hydro_simmim_pretrain_swinv2_base_img256_window16_800ep/ckpt_epoch_670.pth', 'RESUME': '', 'NUM_CLASSES': 1000, 'DROP_RATE': 0.0, 'DROP_PATH_RATE': 0.1, 'LABEL_SMOOTHIN

In [5]:
# load transforms
mean = torch.tensor(config.DATA.MEAN)
std = torch.tensor(config.DATA.STD)
norm = T.Normalize(mean=mean, std=std)

# load model
model = build_model(config, is_pretrain=False)
load_pretrained(config, model, logger)

In [7]:
x = torch.randn(1, 12, 256, 256)
x = norm(x)
embeddings = model.forward_features(x)
print(embeddings.shape)

torch.Size([1, 1024])
